In [125]:
import pandas as pd
data = pd.read_csv('pushkin.csv')[:1000]
data2 = pd.read_csv('dostoewsky.csv')[:1000]

In [126]:
def superrost(words):
    return ''.join(filter(lambda x: ord(x) in range(ord('а'), ord('я')+1) or 
                           ord(x) in range(ord('А'), ord('Я')+1) or 
                           x == ' ', list(words.replace('\n', ' '))))

In [127]:
print('1')
data.text = data.text.apply(superrost)
print('2')
data2.text = data2.text.apply(superrost)

1
2


In [128]:
from nltk.tokenize import TweetTokenizer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

tokenizer = TweetTokenizer()
analyzer = MorphAnalyzer()

def preprocess(text):
    w = text.lower().split()


    filtered_words = [word for word in w if word not in stopwords.words('russian')]

    words = tokenizer.tokenize(' '.join(filtered_words))
    

    for i in range(len(words)):
        words[i] = analyzer.parse(words[i])[0].normal_form
    return ' '.join(words)

In [ ]:
print('1')
data['text'] = data.text.apply(preprocess)
print('2')
data2['text'] = data2.text.apply(preprocess)

1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print('concat')
fitted = pd.concat([data.text,data2.text], axis =0)
print('fitting')
vectorizer.fit(fitted)
print('1')
vectorized = vectorizer.transform(data.text) 
print('2')
vectorized2 = vectorizer.transform(data2.text) 

In [ ]:
minus=(vectorized)-(vectorized2)
minus.shape

In [ ]:
import numpy as np

sums = minus.toarray().sum(axis=0)
sums.shape

In [ ]:
import requests
import json

def beword(word):
    key = 'dict.1.1.20191123T212452Z.411f68444aa87cf0.0625a3c576f183071005d9e3474f48970456140d'
    url = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup?key={}&lang=ru-ru&text={}'.format(key, word)
    answer = requests.get(url)
    if answer.status_code != 200:
        raise Exception('Server err: {}'.format(answer.text))
    if json.loads(answer.text)['def'] == []:
        return False
    else:
        return True


def get_a_word_translation(key: str) -> str:
    URL_AUTH = 'https://developers.lingvolive.com/api/v1.1/authenticate'
    URL_TRANSLATE = 'https://developers.lingvolive.com/api/v1/Minicard'
    KEY = 'OTlkY2Q4MDItZWJlNi00ZWI0LTkyYWYtMThmMTFiNGM2N2FiOjY4MDhkNjVmYTA3MzQ2ZGRiMmQzNDc3YzUyNTA0NTNi'
    
    headers_auth = {'Authorization': 'Basic ' + KEY}
    auth = requests.post(URL_AUTH, headers=headers_auth)
    if auth.status_code == 200:
        token = auth.text
        headers_translate = {
            'Authorization': 'Bearer ' + token
        }
        params = {
            'text': key,
            'srcLang': 1049,
            'dstLang': 1049
        }
        req = requests.get(
            URL_TRANSLATE, headers=headers_translate, params=params)
        res = req.json()
        try:
            value = res['Translation']['Translation']
            return value
        except TypeError:
            if res == 'Incoming request rate exceeded for 50000 chars per day pricing tier':
                raise Exception(res)
            else:
                return None
    else:
        raise Exception('Status code not 200: {}'.format(auth.status_code))


def superbeword(word):
    if get_a_word_translation(word)==None:
        return False
    else:
        return True

In [ ]:
type(sums)

In [ ]:
from time import sleep
li = []
for i in range(len(vectorizer.get_feature_names())):
    word = vectorizer.get_feature_names()[i]
    if (sums[i] > (sums.mean() - 100)) and (len(word)<=25):
        #if (beword(word) or superbeword(word)):
        li.append(word)
li

In [ ]:
len(li)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(sums)